# Sklearn Pipeline for Scoring New Data

In [1]:
import pickle
import pandas as pd

# Import Data
the dataset is available here: kaggle

In [2]:
# load the local dataset 
df = pd.read_csv(
        filepath_or_buffer = '../data/bank_customers_churn_dataset.csv',
        index_col='customer_id'
)

# drop churn column for segmentation
df.drop('churn', axis=1, inplace=True)

# change type of categorical columns "credit_card" and "active_member"
df['credit_card'] = df['credit_card'].apply(lambda x: 'yes' if x == 1 else 'no')
df['active_member'] = df['active_member'].apply(lambda x: 'yes' if x == 1 else 'no')

df.head()

,credit_score,country,gender,age,tenure,balance,products_number,credit_card,active_member,estimated_salary
customer_id,,,,,,,,,,
15634602,619,France,Female,42,2,0.00,1,yes,yes,101348.88
15647311,608,Spain,Female,41,1,83807.86,1,no,yes,112542.58
15619304,502,France,Female,42,8,159660.80,3,yes,no,113931.57
15701354,699,France,Female,39,1,0.00,2,no,no,93826.63
15737888,850,Spain,Female,43,2,125510.82,1,yes,yes,79084.10


# Load Model Artifacts

In [3]:
with open('../artifacts/numeric_imputer.pickle', 'rb') as filename: # trained model to impute missing numeric data
    numeric_imputer = pickle.load(filename)

with open('../artifacts/categorical_imputer.pickle', 'rb') as filename: # trained model to impute missing categorical data
    categorical_imputer = pickle.load(filename) 

with open('../artifacts/rare_encoder.pickle', 'rb') as filename: # trained model to encode rare labels
    rare_encoder = pickle.load(filename)

with open('../artifacts/capper.pickle', 'rb') as filename: # trained model to cap outliers
    capper = pickle.load(filename)   

with open('../artifacts/enc.pickle', 'rb') as filename: # trained one hot encoder
    enc = pickle.load(filename)

with open('../artifacts/model.pickle', 'rb') as filename: # trained random forrest classifier
    model = pickle.load(filename)

# Transform Dataset

In [4]:
# get numeric and categorical columns
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
numeric_columns = df.select_dtypes(include=numerics).columns.to_list()
categorical_columns = df.select_dtypes(exclude=numerics).columns.to_list()

In [5]:
# impute mising numeric features
df_numeric = pd.DataFrame(
    numeric_imputer.transform(df[numeric_columns]), 
    columns=numeric_columns, 
    index=df.index)

# impute mising categorical features
df_categorical = pd.DataFrame(
    categorical_imputer.transform(df[categorical_columns]), 
    columns=categorical_columns, 
    index=df.index)

# concate numeric and categorical features
df = pd.concat([df_numeric, df_categorical], axis=1)

# remove rare labels
df[categorical_columns] = rare_encoder.transform(df[categorical_columns])

# remove outliers
df[numeric_columns] = capper.transform(df[numeric_columns])

# one hot encoding categorical features
df_cat_hotenc = pd.DataFrame(
    enc.transform(df[categorical_columns]), 
    columns=enc.get_feature_names_out(),
    index=df.index) 

# concate numeric and hot-encoded categorical features
df_hotenc = pd.concat([df[numeric_columns], df_cat_hotenc], axis=1)

# predict cluster
labels = model.predict(df_hotenc)

# add cluster label to df
df['cluster'] = labels

df.head()

,credit_score,age,tenure,balance,products_number,estimated_salary,country,gender,credit_card,active_member,cluster
customer_id,,,,,,,,,,,
15634602,619.0,42.0,2.0,0.00,1.0,101348.88,France,Female,yes,yes,4
15647311,608.0,41.0,1.0,83807.86,1.0,112542.58,Spain,Female,no,yes,0
15619304,502.0,42.0,8.0,159660.80,3.0,113931.57,France,Female,yes,no,0
15701354,699.0,39.0,1.0,0.00,2.0,93826.63,France,Female,no,no,0
15737888,850.0,43.0,2.0,125510.82,1.0,79084.10,Spain,Female,yes,yes,0


In [6]:
# Cluster Counts
df['cluster'].value_counts()

0    6493
1    1010
2     947
4     800
3     750
Name: cluster, dtype: int64

In [7]:
# Cluster Centers
pd.concat([
    df.groupby(by='cluster').mean(),
    df.groupby(by='cluster').aggregate(pd.Series.mode)[categorical_columns] ],
    axis=1)

C:\Users\Sebas\AppData\Local\Temp\ipykernel_17160\2551023627.py:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df.groupby(by='cluster').mean(),
C:\Users\Sebas\AppData\Local\Temp\ipykernel_17160\2551023627.py:4: FutureWarning: ['estimated_salary'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  df.groupby(by='cluster').aggregate(pd.Series.mode)[categorical_columns] ],


,credit_score,age,tenure,balance,products_number,estimated_salary,country,gender,credit_card,active_member
cluster,,,,,,,,,,
0,650.432774,39.181884,5.000154,85144.263992,1.534002,100556.750447,Germany,Male,yes,yes
1,652.482178,39.079423,5.096040,59920.970564,1.539454,99342.734149,France,Male,yes,yes
2,649.157339,36.931127,5.139388,64589.570380,1.479707,100604.350391,France,Male,yes,no
3,646.433333,37.689037,4.889333,58629.877200,1.504377,100701.389613,France,Female,yes,no
4,654.305000,39.452074,4.976250,57947.844325,1.517854,96066.118300,France,Female,yes,yes
